In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Load the dataset

df = pd.read_csv('df_UMIK_allSources.csv')
df.head()

,div_and_fold,fold_growth,division_ratio,added_length,generationtime,length_birth,length_final,growth_rate,lineage_ID,generation,start_time,end_time,source
0,1.033597,0.646579,0.502662,1.785882,3.333333,1.690785,3.476667,0.193974,0.0,0.0,0.000000,3.333333,Ace
1,0.870711,0.461032,0.530211,1.183806,2.333333,1.843365,3.027172,0.197585,0.0,1.0,3.500000,5.833333,Ace
2,1.043130,0.633992,0.524340,1.570466,3.500000,1.587268,3.157734,0.181141,0.0,2.0,6.000000,9.500000,Ace
3,1.044470,0.689906,0.470555,1.812272,7.333333,1.485887,3.298159,0.094078,0.0,3.0,9.666667,17.000000,Ace
4,1.071770,0.707084,0.464436,2.003083,8.833333,1.531784,3.534867,0.080047,0.0,4.0,17.166667,26.000000,Ace
